In [99]:
%reload_ext autoreload
%autoreload 2
%xmode Verbose

Exception reporting mode: Verbose


Checking out the basic functions manually

In [100]:
import utils as helper
import pandas as pd

In [129]:
sample_data_raw = [[22,125,13],
               [275,15,3],
               [123,17,3]]
sample_data = pd.DataFrame(sample_data_raw)
print ("oth col is_numeric? :", helper.is_numeric(sample_data, 0))
print (helper.label_counts(sample_data))
print ("###########")
print (helper.most_probable_label(sample_data))
print (helper.most_probable_label(sample_data.iloc[:1, :]))

oth col is_numeric? : True
3     2
13    1
Name: 2, dtype: int64
###########
3
13


General Child Parent relationships below

In [130]:
class Dog():
    def __init__(self):
        pass
    
    def bark(self):
        print ("barking")

In [131]:
class Pup(Dog):
    def __init__(self):
        pass
    def cry(self):
        print ("pups don't cry, steriotyper")

In [132]:
P = Pup()
P.bark()

barking


Checking the criterion class

In [133]:
import criterion

In [134]:
g = criterion.Gini()
curr = g.impurity(sample_data)
print (curr)
print (g.info_gain(sample_data[0:2],
                   sample_data[2:],
                   curr))
# We would get the info gain to be less than .44
# as we did bad job in splitting

0.4444444444444445
0.11111111111111116


Testing Questions.

In [138]:
from question import Question

In [142]:
col = 1
wedge = 17
q = Question(col, wedge)
print("mein col=", col, "mein valu=", wedge, "se >= hai!")
q.divide_on_question(sample_data)[0]

mein col= 1 mein valu= 17 se >= hai!
17 is numeric


,0,1,2
0,22,125,13
2,123,17,3


In [143]:
print("mein col=", col, "mein valu=", wedge, "se < hai!")
q.divide_on_question(sample_data)[1]

mein col= 1 mein valu= 17 se < hai!


,0,1,2
1,275,15,3


Testing some DT Functions

In [144]:
import dt

In [149]:
import criterion
from question import Question
from nodes import DecisionNode, PredictionNode

class DecisionTree:
    """you don't want to instantiate this
    object directly, use a derived class.
    """
    def __init__(self, 
                 method, 
                 max_depth, 
                 max_points):
        self.method = method
        self.max_depth = max_depth
        self.max_points = max_points
        self.criterion = self._make_metric_class()
        self.root = None
        
    
    def train(self, rows):
        pass
#         self.root = self._build_tree(rows, 0)

    def _make_metric_class(self):
        if self.method=='gini':
            return criterion.Gini()

        elif self.method=='entropy':
            return criterion.Entropy()
    
    def _best_split(self, rows):
        '''Finds the best Question (using info_gain) that 
        splits the rows into left and right buckets.

        Returns a Question (records the col_index and the value.)
        @ bad performance when we have really large number of rows.
        Can improve by first sorting and then doing a binary search
        '''
        previous_impurity = self.criterion.impurity(rows)
        BestInfoGain = 0 # find the best col and val to split the rows
        best_question = None
        num_rows, num_cols = rows.shape
        for col in range(num_cols - 1): # last col is the label 
                                       # rows have same len
            for row in range(num_rows):
                val = rows.iloc[row, col]         # this val may be the wedge
                                                  # value for the question
                q = Question(col, val)
                left, right = q.divide_on_question(rows)
                if left.shape[0] == 0 or right.shape[0] == 0:
                    continue           # ignore the case when no splitting
                InfoGain = self.criterion.info_gain(left, right, previous_impurity)
                # if best info gain crossed save state or save Question
                if InfoGain >= BestInfoGain:
                    BestInfoGain = InfoGain
                    best_question = q
        return BestInfoGain, best_question
    
    
    def _build_tree(self, rows, depth):
        '''This function would be recursively called
        We first set the base case where we check if the
        the depth of tree is less than some number or
        we don't have any difference in labels
        @add support for regression
        '''
        if depth == self.max_depth: # forced leaf
            return PredictionNode(rows)

        gain, question = self._best_split(rows)
        if gain == 0:               # we have encountered a leaf
            return PredictionNode(rows)

        left, right = divide_on_question(rows, question)
        left_branch = self._build_tree(left, depth + 1)
        right_branch = self._build_tree(right, depth + 1)
        return DecisionNode(question, left_branch, right_branch)


class DecisionTreeClassifier(DecisionTree):
    """Instantiate this class for a Decision
    Tree for classification tasks.
    """
    def __init__(self,
                 method="gini",
                 max_depth=2,
                 max_points=None):
        super().__init__(method,
                         max_depth,
                         max_points)
    

    def predict(self, row):
        return classify(row, self.root)


In [150]:
d = DecisionTreeClassifier()

In [151]:
d.train(sample_data)

In [154]:
out = d._best_split(sample_data)
print (out)
left, right = out[1].divide_on_question(sample_data)

(0.4444444444444445, <question.Question object at 0x7f0884407ef0>)
left     0    1   2
0  22  125  13
Right      0   1  2
1  275  15  3
2  123  17  3


In [155]:
print ("left")
left

left


,0,1,2
0,22,125,13


In [156]:
print ("right")
right

right


,0,1,2
1,275,15,3
2,123,17,3


Test the nodes that are used to build the DT

In [99]:
import nodes

In [ ]:
nodes.DecisionNode('q', 'r', r)